In [1]:
import numpy as np
import pandas as pd


import seaborn as sns
import matplotlib.pyplot as plt

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\markp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Train dataset
train_set = pd.read_csv("Corona_NLP_train.csv",encoding="latin-1")
train_set_Copy = train_set.copy()

#test dataset
test_set=pd.read_csv('Corona_NLP_test.csv')
test_set_Copy = train_set.copy()

In [3]:
train_set.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
test_set.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [5]:
print("    Train dataset info: \n")
train_set.info()

print("---------------------------\n\n    Test dataset info: \n")
test_set.info()

    Train dataset info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB
---------------------------

    Test dataset info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       3798 non-null   int64 
 1   ScreenName     3798 non-null   int64 
 2   Location       2964 non-null   object
 3   TweetAt        3798 non-null   object
 4   OriginalTweet  3798 non-null   object
 5   Sentim

In [6]:
# count the number of duplicate rows
num_duplicates = train_set.duplicated().sum()

print(f"There are {num_duplicates} duplicate rows in the DataFrame.")

There are 0 duplicate rows in the DataFrame.


In [7]:
train_set = train_set.loc[:, ['Location', 'TweetAt', 'OriginalTweet','Sentiment']]
train_set.head()

,Location,TweetAt,OriginalTweet,Sentiment
0,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [8]:
# Find unique values and their frequencies in the Location column
unique_value_counts = pd.DataFrame({
    'Location': train_set['Location'].value_counts()})

# Sort by descending frequency
unique_value_counts = unique_value_counts.sort_values(by=['Location'], ascending=False)

# Create bar chart of top 20 frequent values in the Location column
top_20 = unique_value_counts.head(20)
top_20.plot.bar(rot=45, figsize=(12, 6), align='edge')  # Rotate x-axis labels by 45 degrees
plt.title("Top 20 Tweet Locations by Frequency")
plt.show()

ValueError: 'Location' is both an index level and a column label, which is ambiguous.

In [ ]:
# define the order of the sentiment categories
sentiment_order = ["Extremely Negative", "Negative", "Neutral", "Positive", "Extremely Positive"]

# create a color palette from red to green
colors = sns.color_palette("RdYlGn", len(sentiment_order))

# set the color map for the barplot
color_map = dict(zip(sentiment_order, colors))

# count the number of instances for each sentiment category
sentiment_counts = train_set["Sentiment"].value_counts()

# create a figure and axis object
fig, ax = plt.subplots(figsize=(10, 6))

# create a bar plot using seaborn with the desired order and color map
sns.set(style="darkgrid")
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, order=sentiment_order, ax=ax, palette=color_map)

# set plot title and axis labels
ax.set_title("Sentiment Distribution in Train Set")
ax.set_xlabel("Sentiment Category")
ax.set_ylabel("Number of Tweets")

# rotate x-axis labels
ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
def choosingcol(df):
    new_df=df.loc[:, ['OriginalTweet', 'Sentiment']]
    return new_df

#------------------------------------------------------------------
def reducing_sentiment(df):
    df = df.loc[df['Sentiment'] != 'Neutral']

    # replace extremely positive and extremely negative with positive and negative, respectively
    df['Sentiment'] = df['Sentiment'].replace({'Extremely Positive': 'Positive', 'Extremely Negative': 'Negative'})

    return df
#-----------------------------------------------------------------
def converting_tonumeric(df):
    sentiment_dict = {'Positive': 1, 'Negative': 0}
    df['Sentiment'] = df['Sentiment'].replace(sentiment_dict)
    return df

In [ ]:
# Using functions to make train_set ready
train_set = choosingcol(train_set)
train_set = reducing_sentiment(train_set)
train_set = converting_tonumeric(train_set)
train_set = train_set.reset_index(drop=True)
train_set

In [ ]:
# Count the number of occurrences of each sentiment value
sentiment_counts = train_set["Sentiment"].value_counts()

# Create a pie chart
labels = [ 'Positive','Negative']
colors = [(1, 0, 0, 0.5), (0, 1, 0, 0.5)]

plt.figure(figsize=(5, 5))
plt.pie(sentiment_counts, labels=labels, colors=colors, autopct='%1.1f%%')

plt.title('Sentiment Distribution')
plt.show()

In [ ]:
contractions_dict = {
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'll": "he shall / he will",
    "he's": "he is",
    "I'd": "I had / I would",
    "I'll": "I shall / I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'll": "it shall / it will",
    "it's": "it is",
    "let's": "let us",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she had / she would",
    "she'll": "she shall / she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they had / they would",
    "they'll": "they shall / they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'll": "we shall / we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who had / who would",
    "who'll": "who shall / who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "you'd": "you had / you would",
    "you'll": "you shall / you will",
    "you're": "you are",
    "you've": "you have"
}

In [ ]:
def cleaning_with_re(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?://(?:www\.[^\s\n\r]+|[^\s\n\r]+)', '', tweet)

    # remove hashtag (#)
    tweet = re.sub(r'#', '', tweet)

    #replaces newline (\n) and carriage return (\r) characters in a tweet with an empty string
    tweet = re.sub(r'[\n\r]', '', tweet)

    #replaces the numbers with an empty string
    tweet = re.sub(r'\d+', '', tweet)

    return tweet

#----------------------------------------------------------------
def expanding_words(tweet):

    words = tweet.split()
    expanded_words = []
    for word in words:
        if word.lower() in contractions_dict:
            expanded_words.extend(contractions_dict[word.lower()].split("/"))
        else:
            expanded_words.append(word)
    return " ".join(expanded_words)

#----------------------------------------------------------------
def tokenizing(tweet):

    stopwords_english = stopwords.words('english')
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    new_tweet = tokenizer.tokenize(tweet)

    clean_tweet=[]

    for word in new_tweet: # Going through every word in tokens list
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation and stopwords
            clean_tweet.append(word)

    return clean_tweet

#----------------------------------------------------------------
def stemming(tweet):
    # Instantiate stemming class
    stemmer = PorterStemmer()

    # Create an empty list to store the stems
    stemmed_tweets = []

    for word in tweet:
        stem_word = stemmer.stem(word)  # stemming word
        stemmed_tweets.append(stem_word)  # append to the list

    return stemmed_tweets

In [ ]:
def preprocessing(tweet):

    tweet=cleaning_with_re(tweet)
    tweet=expanding_words(tweet)
    tweet=tokenizing(tweet)
    tweet=stemming(tweet)

    return tweet

In [ ]:
def build_freqs(listoftweets, labels):

    labelslist = np.squeeze(labels).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets and all the words in them
    freqs = {}
    for y, tweet in zip(labels, listoftweets):
        t=preprocessing(tweet)
        for word in t:
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    # Calculate the sum of frequencies for each word
    word_sums = {}
    for (word, y), count in freqs.items():
        if word in word_sums:
            word_sums[word] += count
        else:
            word_sums[word] = count

    # Divide each key of word and class by the sum
    for (word, y), count in freqs.items():
        freqs[(word, y)] /= word_sums[word]

    return freqs

#------------------------------------------------------------------
def extract_features(tweet, freqs, preprocessing=preprocessing):

    # process_tweet tokenizes, stems, and removes stopwords
    word_list = preprocessing(tweet)

    # 2 elements for [ positive, negative] counts
    features = np.zeros(2)

    # loop through each word in the list of words
    for word in word_list:

        # increment the word count for the positive label 1
        features[0] += freqs.get((word, 1.0),0)

        # increment the word count for the negative label 0
        features[1] += freqs.get((word, 0),0)

    features = features[None, :]
    assert(features.shape == (1, 2))
    return features

In [ ]:
%%time
word_frequencies=build_freqs(train_set['OriginalTweet'].tolist(), train_set['Sentiment'].tolist())

In [ ]:
X = np.zeros((len(train_set['Sentiment']), 2))
for i in range(len(train_set['Sentiment'])):
    X[i, :]= extract_features(train_set['OriginalTweet'][i], word_frequencies)

print(X)

# training labels corresponding to X
Y = np.array(train_set['Sentiment'])

In [ ]:
# Model
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))  # Hidden layer with 4 neurons
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Output layer with logistic regression

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=60, batch_size=32)

In [ ]:
test_set=choosingcol(test_set)
test_set=reducing_sentiment(test_set)
test_set=converting_tonumeric(test_set)
test_set=test_set.reset_index(drop=True)
test_set

In [ ]:
X_test = np.zeros((len(test_set['Sentiment']), 2))
for i in range(len(test_set['Sentiment'])):
    X_test[i, :]= extract_features(test_set['OriginalTweet'][i], word_frequencies)
print(X_test)

In [ ]:
y_prob = model.predict(X_test)
y_pred = np.round(y_prob).flatten()
y_test = test_set['Sentiment'].tolist()

# Evaluate the model on the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test set accuracy = {accuracy}")
print(f"Test set precision = {precision}")
print(f"Test set recall = {recall}")
print(f"Test set F1 score = {f1}")

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix
cm_norm = cm / np.sum(cm)

# Set the figure size
plt.figure(figsize=(8, 6))

# Create the heatmap
plt.imshow(cm_norm, cmap=plt.cm.Blues)
plt.xticks([0, 1], ['Negative', 'Positive'])
plt.yticks([0, 1], ['Negative', 'Positive'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.colorbar()

# Display the percentages in the plot
for i in range(cm_norm.shape[0]):
    for j in range(cm_norm.shape[1]):
        plt.text(j, i, '{:.1%}'.format(cm_norm[i, j]), ha='center', va='center')

# Remove the grid lines
plt.grid(False)
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Compute the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Compute the AUC score
auc_score = roc_auc_score(y_test, y_prob)

# Plot the ROC curve
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()